Libraries import 

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


Pick location from Nominatim for Toronto canada

In [5]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.653963, -79.387207.


Load data from already stored in the CSV

In [6]:
toronto_data_complete=pd.read_csv('Toronto_lat_long.csv')
toronto_data_complete.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Filter and load it in to new data frame

In [7]:
toronto_data_complete['TorontoExist']=toronto_data_complete['Neighborhood'].str.find('Toronto')
toronto_data=toronto_data_complete[toronto_data_complete['TorontoExist']>-1].reset_index(drop=True)
toronto_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,TorontoExist
0,M3K,North York,"CFB Toronto,Downsview East",43.737473,-79.464763,4
1,M4J,East York,East Toronto,43.685347,-79.338106,5
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,6
3,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752,18
4,M5K,Downtown Toronto,"Design Exchange,Toronto Dominion Centre",43.647177,-79.381576,16


Load the location on the map by using the already loaded 

In [8]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [9]:
CLIENT_ID = 'M1XUTHYN2BJ2IGDSAYF1JMFMSPS4Q51R2WP35IKIGXAGI2SV' # your Foursquare ID
CLIENT_SECRET = '1ULH3MMVPXAPY24A0GCJOQQ3OAC0EHAQVFXKELQUDXH01RCF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: M1XUTHYN2BJ2IGDSAYF1JMFMSPS4Q51R2WP35IKIGXAGI2SV
CLIENT_SECRET:1ULH3MMVPXAPY24A0GCJOQQ3OAC0EHAQVFXKELQUDXH01RCF


In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [11]:
LIMIT=100
Toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

CFB Toronto,Downsview East
East Toronto
North Toronto West
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Harbord,University of Toronto
Humber Bay Shores,Mimico South,New Toronto


In [12]:
print(Toronto_venues.shape)
Toronto_venues.head()

(271, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"CFB Toronto,Downsview East",43.737473,-79.464763,Toronto Downsview Airport (YZD),43.738883,-79.470111,Airport
1,"CFB Toronto,Downsview East",43.737473,-79.464763,Ancaster Park,43.734706,-79.464777,Park
2,"CFB Toronto,Downsview East",43.737473,-79.464763,VAUGHAN GARAGE DOOR REPAIR,43.734877,-79.468584,Other Repair Shop
3,East Toronto,43.685347,-79.338106,The Path,43.683923,-79.335007,Park
4,East Toronto,43.685347,-79.338106,Sammon Convenience,43.686951,-79.335007,Convenience Store


In [12]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"CFB Toronto,Downsview East",3,3,3,3,3,3
"Design Exchange,Toronto Dominion Centre",100,100,100,100,100,100
East Toronto,3,3,3,3,3,3
"Harbord,University of Toronto",34,34,34,34,34,34
"Harbourfront East,Toronto Islands,Union Station",100,100,100,100,100,100
"Humber Bay Shores,Mimico South,New Toronto",15,15,15,15,15,15
North Toronto West,16,16,16,16,16,16


In [13]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 98 uniques categories.


In [16]:
Toronto_venues['Neighborhood'].head()

0    CFB Toronto,Downsview East
1    CFB Toronto,Downsview East
2    CFB Toronto,Downsview East
3                  East Toronto
4                  East Toronto
Name: Neighborhood, dtype: object

In [13]:
# one hot encoding
toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,American Restaurant,Aquarium,Art Gallery,Asian Restaurant,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Store,Bistro,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comfort Food Restaurant,Concert Hall,Convenience Store,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Event Space,Fast Food Restaurant,Food Court,French Restaurant,Fried Chicken Joint,Gastropub,General Travel,Gluten-free Restaurant,Greek Restaurant,Gym,Gym / Fitness Center,History Museum,Hobby Shop,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Lake,Liquor Store,Lounge,Mexican Restaurant,Monument / Landmark,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Other Repair Shop,Park,Performing Arts Venue,Pharmacy,Pizza Place,Plaza,Poutine Place,Pub,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Skating Rink,Smoothie Shop,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"CFB Toronto,Downsview East",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"CFB Toronto,Downsview East",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"CFB Toronto,Downsview East",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,East Toronto,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,East Toronto,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
toronto_onehot.shape

(271, 98)

In [14]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,American Restaurant,Aquarium,Art Gallery,Asian Restaurant,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beer Bar,Beer Store,Bistro,Bookstore,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,Comfort Food Restaurant,Concert Hall,Convenience Store,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Event Space,Fast Food Restaurant,Food Court,French Restaurant,Fried Chicken Joint,Gastropub,General Travel,Gluten-free Restaurant,Greek Restaurant,Gym,Gym / Fitness Center,History Museum,Hobby Shop,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Lake,Liquor Store,Lounge,Mexican Restaurant,Monument / Landmark,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Other Repair Shop,Park,Performing Arts Venue,Pharmacy,Pizza Place,Plaza,Poutine Place,Pub,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shopping Mall,Skating Rink,Smoothie Shop,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Theater,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar
0,"CFB Toronto,Downsview East",0.0000,0.333333,0.000000,0.00,0.00,0.00,0.0000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.0000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.0000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.333333,0.333333,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.00,0.0000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.0000,0.00,0.0000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00
1,"Design Exchange,Toronto Dominion Centre",0.0000,0.000000,0.020000,0.00,0.01,0.01,0.0000,0.040000,0.00,0.030000,0.00,0.01,0.020000,0.000000,0.00,0.010000,0.01,0.00,0.00,0.02,0.070000,0.010000,0.0000,0.01,0.130000,0.000000,0.000000,0.000000,0.01,0.000000,0.00,0.03,0.000000,0.0000,0.00,0.00,0.010000,0.01,0.000000,0.010000,0.03,0.01,0.01,0.01,0.020000,0.0100,0.00,0.000000,0.06,0.01,0.01,0.00,0.040000,0.020000,0.00,0.000000,0.02,0.000000,0.00,0.01,0.00,0.01,0.000000,0.000000,0.01,0.000000,0.000000,0.00,0.000000,0.020000,0.01,0.000000,0.010000,0.0000,0.050000,0.01,0.0000,0.010000,0.00,0.020000,0.01,0.00,0.00,0.01,0.0100,0.01,0.0100,0.00,0.02,0.00,0.000000,0.01,0.010000,0.01,0.01,0.000000,0.01
2,East Toronto,0.0000,0.000000,0.000000,0.00,0.00,0.00,0.0000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.0000,0.00,0.333333,0.000000,0.000000,0.000000,0.00,0.333333,0.00,0.00,0.000000,0.0000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.000000,0.0000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.333333,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0000,0.000000,0.00,0.0000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.0000,0.00,0.0000,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.00
3,"Harbord,University of Toronto",0.0000,0.000000,0.000000,0.00,0.00,0.00,0.0000,0.058824,0.00,0.058824,0.00,0.00,0.029412,0.029412,0.00,0.058824,0.00,0.00,0.00,0.00,0.117647,0.029412,0.0000,0.00,0.029412,0.029412,0.029412,0.029412,0.00,0.000000,0.00,0.00,0.029412,0.0000,0.00,0.00,0.000000,0.00,0.029412,0.000000,0.00,0.00,0.00,0.00,0.029412,0.0000,0.00,0.000000,0.00,0.00,0.00,0.00,0.058824,0.058824,0.00,0.000000,0.00,0.000000,0.00,0.00,0.00,0.00,0.029412,0.029412,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.029412,0.029412,0.0000,0.058824,0.00,0.0000,0.029412,0.00,0.000000,0.00,0.00,0.00,0.00,0.0000,0.00,0.0000,0.00,0.00,0.

In [15]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----CFB Toronto,Downsview East----
               venue  freq
0            Airport  0.33
1               Park  0.33
2  Other Repair Shop  0.33
3                Pub  0.00
4              Plaza  0.00


----Design Exchange,Toronto Dominion Centre----
                venue  freq
0         Coffee Shop  0.13
1                Café  0.07
2               Hotel  0.06
3          Restaurant  0.05
4  Italian Restaurant  0.04


----East Toronto----
                 venue  freq
0          Coffee Shop  0.33
1    Convenience Store  0.33
2                 Park  0.33
3  Rental Car Location  0.00
4        Poutine Place  0.00


----Harbord,University of Toronto----
                 venue  freq
0                 Café  0.12
1           Restaurant  0.06
2            Bookstore  0.06
3  Japanese Restaurant  0.06
4   Italian Restaurant  0.06


----Harbourfront East,Toronto Islands,Union Station----
                venue  freq
0         Coffee Shop  0.12
1               Hotel  0.05
2            Aquarium  0.05
3  I

In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [25]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 2, 0, 0, 4, 3], dtype=int32)

In [28]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,TorontoExist,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3K,North York,"CFB Toronto,Downsview East",43.737473,-79.464763,4,1,Airport,Park,Other Repair Shop,Wine Bar,Discount Store,College Gym,Comfort Food Restaurant,Concert Hall,Convenience Store,Dance Studio
1,M4J,East York,East Toronto,43.685347,-79.338106,5,2,Coffee Shop,Park,Convenience Store,Wine Bar,College Gym,Comfort Food Restaurant,Concert Hall,Dance Studio,Deli / Bodega,Dessert Shop
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,6,3,Coffee Shop,Yoga Studio,Spa,Fast Food Restaurant,Diner,Mexican Restaurant,Dessert Shop,Park,Clothing Store,Rental Car Location
3,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752,18,0,Coffee Shop,Hotel,Aquarium,Italian Restaurant,Café,Sporting Goods Shop,Pizza Place,Fried Chicken Joint,Restaurant,Brewery
4,M5K,Downtown Toronto,"Design Exchange,Toronto Dominion Centre",43.647177,-79.381576,16,0,Coffee Shop,Café,Hotel,Restaurant,Italian Restaurant,Bakery,Bar,Gastropub,Deli / Bodega,Japanese Restaurant


In [2]:
# create map
import folium
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

NameError: name 'latitude' is not defined